<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
#Start Here

import pandas as pd

PATH = "/Users/ekselan/Desktop/LAMBDA/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv"
reviews = pd.read_csv(PATH)
print(reviews.shape)
reviews.head()

(28332, 24)


,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [4]:
df = reviews.copy()
df['brand'] = df['brand'].apply(lambda x: x.lower())
df['brand'].value_counts()

amazon          16153
amazonbasics    12179
Name: brand, dtype: int64

In [6]:
df['reviews.text'][100]

"As a teacher, I need tons of batteries, but I refused to spend excessive amounts on them, so I figured this was the best option! They are long-lasting, worth the money, cute (if that even matters, lol)! You won't find a deal like this in stores!I highly recommend!"

## Tokenization

In [7]:
import spacy

nlp = spacy.load("en_core_web_lg")

In [90]:
samp = df.sample(100)

In [89]:
# Add stop words

STOP_WORDS = nlp.Defaults.stop_words.union(["Amazon",
                                            "Christmas",
                                            "Kindle",
                                            "fire",
                                            "tablet",
                                            "device",
                                            "battery",
                                            "Fire"])

In [95]:
## Use Lemmas as our tokens

tokens = []

for doc in nlp.pipe(df['reviews.text']):
    
    doc_tokens = []
    
    for token in doc:
        if (token.is_stop == False) & (token.is_punct == False) & (token.pos != 'PRON') & (token not in STOP_WORDS):
            doc_tokens.append(token.lemma_.strip())
            
    tokens.append(doc_tokens)

In [96]:
# Confirmation that shape is correct

len(tokens) == df.shape[0]

True

## Gensim LDA Topic Modeling

In [98]:
import gensim

from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore

In [99]:
id2word = corpora.Dictionary(tokens)

In [100]:
len(id2word.keys())

9727

In [101]:
id2word.filter_extremes(no_below=5, no_above=.95)

In [102]:
len(id2word.keys())

3087

In [103]:
corpora = [id2word.doc2bow(doc) for doc in tokens]

In [104]:
lda = LdaMulticore(corpus=corpora, # stream of document vectors
                   id2word=id2word, # mapping from IDs to words, used to determine vocab size
                   num_topics=15, # n_requested latent topics to be extracted from training corpus
                   passes=100,
                   workers=5) # If none, all avail cores are used. Optimal:set to real n of cores -1




In [86]:
lda.print_topics()

[(0,
  '0.049*"tablet" + 0.037*"Fire" + 0.029*"Amazon" + 0.026*"fire" + 0.023*"8" + 0.019*"buy" + 0.019*"$" + 0.016*"7" + 0.016*"iPad" + 0.014*"screen"'),
 (1,
  '0.123*"old" + 0.107*"year" + 0.054*"love" + 0.033*"buy" + 0.032*"tablet" + 0.031*"2" + 0.024*"3" + 0.021*"control" + 0.019*"4" + 0.019*"get"'),
 (2,
  '0.091*"great" + 0.081*"tablet" + 0.063*"good" + 0.059*"price" + 0.049*"nice" + 0.036*"value" + 0.028*"fast" + 0.028*"size" + 0.023*"use" + 0.020*"easy"'),
 (3,
  '0.182*"battery" + 0.071*"long" + 0.062*"brand" + 0.057*"good" + 0.051*"price" + 0.035*"great" + 0.026*"buy" + 0.026*"life" + 0.025*"cheap" + 0.015*"time"'),
 (4,
  '0.050*"Kindle" + 0.040*"read" + 0.033*"easy" + 0.031*"book" + 0.029*"light" + 0.025*"screen" + 0.022*"kindle" + 0.020*"reader" + 0.016*"use" + 0.016*"like"'),
 (5,
  '0.081*"app" + 0.051*"tablet" + 0.024*"kid" + 0.024*"Amazon" + 0.021*"like" + 0.019*"amazon" + 0.018*"free" + 0.017*"need" + 0.015*"store" + 0.013*"Google"'),
 (6,
  '0.091*"battery" + 0.023*

In [87]:
import re
words = [re.findall('"([^"]*)"',t[1]) for t in lda.print_topics()]
topics = [' '.join(t[0:5]) for t in words]

In [88]:
for id, t in enumerate(topics):
    print(f"-----Topic {id} -----")
    print(t, end="\n\n")

-----Topic 0 -----
tablet Fire Amazon fire 8

-----Topic 1 -----
old year love buy tablet

-----Topic 2 -----
great tablet good price nice

-----Topic 3 -----
battery long brand good price

-----Topic 4 -----
Kindle read easy book light

-----Topic 5 -----
app tablet kid Amazon like

-----Topic 6 -----
battery Amazon box order pack

-----Topic 7 -----
kid easy use tablet love

-----Topic 8 -----
game play book tablet watch

-----Topic 9 -----
charge battery buy time power

-----Topic 10 -----
love buy gift purchase Christmas

-----Topic 11 -----
battery day use month 2

-----Topic 12 -----
device Amazon use Alexa sound

-----Topic 13 -----
great good product price recommend

-----Topic 14 -----
work great far fine expect



## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling